In [1]:
%pip install sentence-transformers

  Using cached threadpoolctl-3.3.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/132.8 kB ? eta -:--:--
   --------- ----------------------------- 30.7/132.8 kB 660.6 kB/s eta 0:00:01
   ------------------------------------- -- 122.9/132.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 132.8/132.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.6 MB 12.6 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/10.6 MB 10.5 MB/s eta 0:00:01
   -------- ------------------------------- 2.4/10.6 MB 16.9 MB/s eta 0:00:01
   --------------- ------------------------ 4.2/10.6 MB 22.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.1/10.6 MB 25.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.3/10.6 MB 29.4 MB/s eta 0:00:01
   -------------------------------------- - 10.3/10.6 MB 31.2 MB/s eta 0:00:01
   

In [1]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_transformers import LongContextReorder
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI

# Get embeddings.
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

texts = [
    "Basquetball is a great sport.",
    "Fly me to the moon is one of my favourite songs.",
    "The Celtics are my favourite team.",
    "This is a document about the Boston Celtics",
    "I simply love going to the movies",
    "The Boston Celtics won the game by 20 points",
    "This is just a random text.",
    "Elden Ring is one of the best games in the last 15 years.",
    "L. Kornet is one of the best Celtics players.",
    "Larry Bird was an iconic NBA player.",
]

# Create a retriever
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)
query = "What can you tell me about the Celtics?"

# Get relevant documents ordered by relevance score
docs = retriever.get_relevant_documents(query)
docs

[Document(page_content='This is a document about the Boston Celtics'),
 Document(page_content='The Celtics are my favourite team.'),
 Document(page_content='L. Kornet is one of the best Celtics players.'),
 Document(page_content='The Boston Celtics won the game by 20 points'),
 Document(page_content='Larry Bird was an iconic NBA player.'),
 Document(page_content='Elden Ring is one of the best games in the last 15 years.'),
 Document(page_content='Basquetball is a great sport.'),
 Document(page_content='I simply love going to the movies'),
 Document(page_content='Fly me to the moon is one of my favourite songs.'),
 Document(page_content='This is just a random text.')]

In [4]:
# Reorder the documents:
# Less relevant document will be at the middle of the list and more
# relevant elements at beginning / end.
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[Document(page_content='The Celtics are my favourite team.'),
 Document(page_content='The Boston Celtics won the game by 20 points'),
 Document(page_content='Elden Ring is one of the best games in the last 15 years.'),
 Document(page_content='I simply love going to the movies'),
 Document(page_content='This is just a random text.'),
 Document(page_content='Fly me to the moon is one of my favourite songs.'),
 Document(page_content='Basquetball is a great sport.'),
 Document(page_content='Larry Bird was an iconic NBA player.'),
 Document(page_content='L. Kornet is one of the best Celtics players.'),
 Document(page_content='This is a document about the Boston Celtics')]

In [5]:
# We prepare and run a custom Stuff chain with reordered docs as context.

# Override prompts
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
llm = OpenAI()
stuff_prompt_override = """Given this text extracts:
-----
{context}
-----
Please answer the following question:
{query}"""
prompt = PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Instantiate the chain
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=reordered_docs, query=query)

c:\Users\karol\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"\nSome potential answers based on the given text extracts could be:\n\n- The Celtics are someone's favorite team.\n- The Boston Celtics won a game by 20 points.\n- L. Kornet is one of the best Celtics players.\n- This is a document about the Boston Celtics.\n- Larry Bird was an iconic NBA player.\n- The Boston Celtics are a basketball team.\n- The Boston Celtics are mentioned in multiple texts.\n- The Celtics have a player named L. Kornet.\n- The Celtics have a connection to the city of Boston.\n- The Celtics won a game by 20 points in the past."